# Cohort Project - DWave Data Set - Work Notebook

Three main sections:

> 1) Data loader for the Cohort Project DWave data files provided. No solver code implemented yet.

> 2) Sanity check of the DWave ocean library

> 3) An attempt at implementing a H2 ground state calculator leveraging the paper arXiv:1611.01068v1 [quant-ph] 3 Nov 2016







In [1]:
!pip install dwave-ocean-sdk

  Created wheel for pyqubo: filename=pyqubo-0.4.0-py3-none-any.whl size=40716 sha256=25fce43bf0500ea83b8379a82004d5d75bb2c95a29d9ce5736f230d6a1935c1b
  Stored in directory: c:\users\jmalliaros\appdata\local\pip\cache\wheels\9a\bf\48\609adb825c687d2d29e066909309a779039cd83fd8fd1244ad
Successfully built pyqubo


In [2]:
# Copy files locally from a google drive account in Colab
# 
# SET THE PATH APPLICABLE TO YOUR LOCAL ENVIRONMENT
#
# 1 - Mount your Google drive in Colab
# 2 - Set the path below
# 3 - Test without the cp line first
# 4 - Comment out the cp command line after verifying that the ls was successful

path = "drive/'My Drive'/Quantum-CDL/DWaveDataSet/IsingSamplesDW"
!ls $path/*
#!cp -r $path/* .

path2 = "drive/'My Drive'/Quantum-CDL/DWaveDataSet/IsingHamiltonian"
!ls $path2/*
#!cp -r $path2/* .



'ls' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import numpy as np
import helper as dw
import csv 

ModuleNotFoundError: No module named 'helper'

## Load the Training and Validation data from the DWave npz file


In [4]:
train, val = dw.load_dataset("dataset_x1a46w3557od23s750k9.npz")

NameError: name 'dw' is not defined

In [24]:
print(train[0])
print(val[0])

print (train.shape)
print (val.shape)

[0 0 1 ... 1 1 0]
[0 0 1 ... 1 1 0]
(40000, 2041)
(10000, 2041)


## Load the correlated features csv data

In [28]:
corr_feat = []
with open('correlated_features.csv', newline='') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  for row in reader:
    corr_feat.append(row)

print (len(corr_feat))

#for r in corr_feat:
#  print (r[0],r[1],r[2])



5975


## DWave library check

Note: Libraries used do not actually run on the QPU but uses Tabu sampler library


In [ ]:
import networkx as nx
from collections import defaultdict
from dimod import BinaryQuadraticModel
from tabu import TabuSampler
import dimod

In [ ]:
# Sanity check using TabuSampler (not on the QPU)

response = TabuSampler().sample_ising({'a': -0.5, 'b': 1.0}, {('a', 'b'): -1})

print(response)




   a  b energy num_oc.
0 -1 -1   -1.5       1
['SPIN', 1 rows, 1 samples, 2 variables]


In [ ]:
Q = defaultdict(int)
sampler = TabuSampler()
offset = 0

# Solve y = -5 xa -3 xb -8 xc -6 xd + 4 xaxb + 8 xaxc + 2 xbxc + 10 xcxd
# Answer should be -11 for qubits 1, 0, 0, 1

Q = [(-6, 5, 0, 0), (5,-8,1,4), (0,1,-3,2), (0,4,2,-5)]

bqm = BinaryQuadraticModel.from_qubo(Q, offset=offset)
sampleset = sampler.sample(bqm)
print(sampleset)
print(sampleset.first.energy)


   0  1  2  3 energy num_oc.
0  1  0  0  1  -11.0       1
['BINARY', 1 rows, 1 samples, 4 variables]
-11.0


# Experimenting for H2 Energy Calculations

## Implementing Hamiltonian for calculating H2 Energies








### Work in progress - Status

At this stage the results do not match the target exact numbers. The code will need reviewing. One peculiar outcome is that results obtained for row N strangely coincide with the target on row N+1. 

See results the "Process" header further below.


[Description 1](https://drive.google.com/file/d/18dx3Pkg_2QKfJL6LLsaTuAQoMfofCrpN/view?usp=sharing)

[Description 2](https://drive.google.com/file/d/1clVgCZbR9aieIrd4SDvwIxwzI7t_el8O/view?usp=sharing)

We can use this symmetry to reduce the
Hamiltonian to the following effective Hamiltonian, acting only on two qubits:
HH2 = g01 + g1σ0
z + g2σ1
z + g3σ0
zσ1
z + g4σ0
xσ1
x + g4σ0
yσ1
y
= g01 + H0
(4)
H0 = g1σ0
z + g2σ1
z + g3σ0
zσ1
z + g4σ0
xσ1
x + g4σ0
yσ1
y (5)
By squaring the Hamiltonian H0 and modifying it, one can get a new Ising Hamiltonian:



H1 = H2
0 + 2g3H0 = 

a1 + a2(σ0
z + σ1
z ) + a3σ0
zσ1
z (6)

With:
a1 = g2
1 + g2
2 + g2
3 + 2g2
4; a2 = 2(g1 + g2)g3; a3 = 2(g1g2 − g2
4 + g2
3) (7)

Here we present steps to get the ground state of HH2 by using the new Ising Hamiltonian H1 (Eq.6).
1. If |g1| + |g2| + |g4| < |g3| start computing by H1 and get the result Y . Otherwise increase |g3| by
|g1| + |g2| + |g4| and start computing.
2. Solve equation x2 + 2g3x = Y and get σ1
x and σ2
x (σ1
x <= σ2
x). Add |g1| + |g2| + |g4| to σ1
x if added to g3
before (we just assume g3 > 0.) Compare σ1
x with g3 − g1 − g2 (or g3 + g1 + g2) to get the ground state of
H0. Add g0 to get the ground state of HH2 .


## Class for H2 energy calculation

Loads the table file included in the source paper.
Calculates the ground state energy given one set of parameters (one row)
from the table

In [ ]:


class H2_Energy:

  def data(self):
    return self.coeff

  # Initialize with coefficient data
  def __init__(self,file):

    self.coeff = []
    with open(file, newline='') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      first = True
      for row in reader:
        if ( first ): 
          first = False
          continue
        row = list(map(float, row)) 
        self.coeff.append(row)

  # Solve the energy for the given radius and coefficients

  def solve(self,R,g0,g1,g2,g3,g4,samples,exact):

    print("\nSolving for R=%f (exact=%f)" % (R,exact),g0,g1,g2,g3,g4)

    # Step 1: If |g1| + |g2| + |g4| >= |g3| : g3 += |g1| + |g2| + |g4|
    # Remember the g3 addition for later

    g3_addition = 0
    if ( abs(g1) + abs(g2) + abs(g4) >= abs(g3)) :
      g3_addition = (abs(g1)+abs(g2)+abs(g4))
      # g3 = abs(g3) + g3_addition
      g3 = (abs(g3) + g3_addition)


    # Step 1b: calculate a1, a2, a3
    # a1 = g1**2 + g2**2 + g3**2 + 2 * g4**2
    # a2 = 2 * ( g1 + g2) * g3
    # a3 = 2 * (g1*g2 - g4**2 + g3**2)

    a1 = g1**2 + g2**2 + g3**2 + 2 * g4**2
    a2 = 2 * ( g1 + g2) * g3
    a3 = 2 * (g1*g2 - g4**2 + g3**2)

    # Step 2 : Solve H0**2 + 2g3H0 = a1 + a2( sz0 + sz1 ) + a3 (sz0*sz1) = Y
    # (the squared H0 hamiltonian)
    # where sz0 is sigma z bit 0, ...
    # Convert szi -> (2 xi - 1) (ising to qubo conversion)
    # then simplify the equation to obtain the quadratic coeeficients of a two qubit system
    # Calculated Q matrix:
    #   |  (2a2-2a3)    2a3     |
    #   |     2a3    (2a2-2a3)  |

    Q=defaultdict(float)
    Q[0,0] = 2 * a2 - 2 * a3
    Q[0,1] = 4 * a3
    Q[1,0] = 4 * a3
    Q[1,1] = 2 * a2 - 2 * a3 

    #Q = [(2 * a2 - 2 * a3, 2 * a3),(2 * a3,2 * a2 - 2 * a3 )]

    #print(Q)

    # Solve the equation. First solution is lowest energy

    offset = 0
    bqm = BinaryQuadraticModel.from_qubo(Q, offset=offset)
    sampleset = sampler.sample(bqm, num_reads = samples)
    #print(sampleset.first.sample.keys() )
    #print(sampleset.first.energy)
    print(sampleset.first.sample)

    # Step 3: Get x0 and x1 for first energy result

    for set in sampleset.data():
      x0 = set.sample[0]
      x1 = set.sample[1]
      energy = set.energy
      print("x0,x1,ener : ",x0,x1,energy)
      break

    # Back to ising 

    sz0 = ( 2 * x0 ) -1 
    sz1 = ( 2 * x1 ) -1 

    # Calculate Y1 using x0x1 for verification : Y and Y1 match

    #Y1 = 4 * x0 * x1 + (2*a2 - 2*a3) * x0 + (2*a2-2*a3) * x1 + a3 - 2*a2 + a1

    # Step 4: Calculate Y = ( a1 + a2( sz0 + sz1 ) + a3 (sz0*sz1))

    Y = ( a1 + a2 * ( sz0 + sz1 ) + a3 * (sz0 * sz1))

    # Get hx1 and hx2 in :
    # hx**2 + 2*g3*hx = Y
    # a = 1, b = 2g3, c = -Y

    a = 1
    b = 2 * g3
    c = -Y

    # Solve H1 for x (minimum of the two possibilities)

    hx1 = ( -b + np.sqrt(b**2-4*a*c)) / (2 * a ) 
    hx2 = ( -b - np.sqrt(b**2-4*a*c)) / (2 * a ) 

    #print("hx1,hx2",hx1,hx2)

    if ( hx2 < hx1):
      swp = hx2
      hx2 = hx1
      hx1 = swp

    # Add g3_addition to hx1

    hx1 += g3_addition

    # H0 = g1σ0 z + g2σ1 z + g3σ0 zσ1 z +
  
    H0_ver = (g1 * sz0) + (g2 * sz1) + (g3 * sz0 * sz1) 

    H0 = hx1

    H = H0 + g0

    return (H)



H2 = H2_Energy('H2_coefficients_exact_simulated.csv')
print(H2.data())

#'H2_coefficients_exact_simulated.csv'


[[0.6, 1.5943, 0.5132, -1.1008, 0.6598, 0.0809, -0.5703, -0.5703], [0.65, 1.4193, 0.5009, -1.0366, 0.6548, 0.0813, -0.6877, -0.6877], [0.7, 1.2668, 0.4887, -0.9767, 0.6496, 0.0818, -0.7817, -0.7817], [0.75, 1.1329, 0.4767, -0.9208, 0.6444, 0.0824, -0.8575, -0.8575], [0.8, 1.0144, 0.465, -0.8685, 0.639, 0.0829, -0.9188, -0.9188], [0.85, 0.909, 0.4535, -0.8197, 0.6336, 0.0835, -0.9685, -0.9685], [0.9, 0.8146, 0.4422, -0.774, 0.6282, 0.084, -1.0088, -1.0088], [0.95, 0.7297, 0.4313, -0.7312, 0.6227, 0.0846, -1.0415, -1.0415], [1.0, 0.6531, 0.4207, -0.691, 0.6172, 0.0852, -1.0678, -1.0678], [1.05, 0.5836, 0.4103, -0.6533, 0.6117, 0.0859, -1.0889, -1.0889], [1.1, 0.5204, 0.4003, -0.6178, 0.6061, 0.0865, -1.1056, -1.1056], [1.15, 0.4626, 0.3906, -0.5843, 0.6006, 0.0872, -1.1186, -1.1186], [1.2, 0.4098, 0.3811, -0.5528, 0.5951, 0.0879, -1.1285, -1.1285], [1.25, 0.3613, 0.372, -0.523, 0.5897, 0.0886, -1.1358, -1.1358], [1.3, 0.3167, 0.3631, -0.4949, 0.5842, 0.0893, -1.1409, -1.1409], [1.35, 0.2

## Process the table and calculate the H2 ground states


In [ ]:
data = H2.data()


for row in range(len(data)):
  R,g0,g1,g2,g3,g4,exact,sim = data[row]
  print(R,g0,g1,g2,g3,g4)
  HH2 = H2.solve(R,g0,g1,g2,g3,g4, 25, exact )
  print("HH2 = %f, Exact = %f" % (HH2,exact))


0.6 1.5943 0.5132 -1.1008 0.6598 0.0809

Solving for R=0.600000 (exact=-0.570300) 1.5943 0.5132 -1.1008 0.6598 0.0809
{0: 1, 1: 0}
x0,x1,ener :  1 0 -25.427033759999997
HH2 = -0.687590, Exact = -0.570300
0.65 1.4193 0.5009 -1.0366 0.6548 0.0813

Solving for R=0.650000 (exact=-0.687700) 1.4193 0.5009 -1.0366 0.6548 0.0813
{0: 1, 1: 0}
x0,x1,ener :  1 0 -23.445527400000003
HH2 = -0.781574, Exact = -0.687700
0.7 1.2668 0.4887 -0.9767 0.6496 0.0818

Solving for R=0.700000 (exact=-0.781700) 1.2668 0.4887 -0.9767 0.6496 0.0818
{0: 0, 1: 1}
x0,x1,ener :  0 1 -21.655856440000004
HH2 = -0.857304, Exact = -0.781700
0.75 1.1329 0.4767 -0.9208 0.6444 0.0824

Solving for R=0.750000 (exact=-0.857500) 1.1329 0.4767 -0.9208 0.6444 0.0824
{0: 1, 1: 0}
x0,x1,ener :  1 0 -20.041267999999995
HH2 = -0.918683, Exact = -0.857500
0.8 1.0144 0.465 -0.8685 0.639 0.0829

Solving for R=0.800000 (exact=-0.918800) 1.0144 0.465 -0.8685 0.639 0.0829
{0: 0, 1: 1}
x0,x1,ener :  0 1 -18.573192600000002
HH2 = -0.968368, 